# PositionEmbedding Layer

In [ ]:
class PositionEmbedding(layers.Layer):
    def __init__(self, initializer="glorot_uniform", **kwargs ):
        super().__init__(**kwargs)
        self._initializer = tf.keras.initializers.get(initializer)

    def get_config(self):
        config = {
            "initializer": tf.keras.initializers.serialize(self._initializer),
        }
        base_config = super(PositionEmbedding, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def build(self, input_shape):
        sequence_length = input_shape[-2]
        width = input_shape[-1]

        self._position_embeddings = self.add_weight(
            "position_embeddings",
            shape=[sequence_length, width],
            initializer=self._initializer)

        super().build(input_shape)

    def call(self, inputs):
        return inputs + self._position_embeddings

xx = tf.zeros((2, 4, 12))
l = PositionEmbedding()
output = l(xx)

print('PositionEmbedding Layer')
print('Input: {} --> {}'.format(xx.shape, output.shape))
print(f'Embedding Size: {l.weights[0].shape}')
print(f'Verify Embeddings: {tf.reduce_all(tf.math.equal(output, l.weights))}')

# StochasticDepth Layer

In [ ]:
"""It is sourced from tensorflow-models package

Source: https://github.com/tensorflow/models/blob/v2.12.0/official/vision/modeling/layers/nn_layers.py#L227-L262
"""
class StochasticDepth(layers.Layer):
    """Creates a stochastic depth layer."""

    def __init__(self, stochastic_depth_drop_rate, **kwargs):
        """Initializes a stochastic depth layer.

        Args:
          stochastic_depth_drop_rate: A `float` of drop rate.
          **kwargs: Additional keyword arguments to be passed.

        Returns:
          A output `tf.Tensor` of which should have the same shape as input.
        """
        super(StochasticDepth, self).__init__(**kwargs)
        self._drop_rate = stochastic_depth_drop_rate

    def get_config(self):
        config = {'stochastic_depth_drop_rate': self._drop_rate}
        base_config = super(StochasticDepth, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def call(self, inputs, training=None):
        if training is None:
            training = tf.keras.backend.learning_phase()
        if not training or self._drop_rate is None or self._drop_rate == 0:
            return inputs

        keep_prob = 1.0 - self._drop_rate
        batch_size = tf.shape(inputs)[0]
        random_tensor = keep_prob
        random_tensor += tf.random.uniform(
            [batch_size] + [1] * (inputs.shape.rank - 1), dtype=inputs.dtype)
        binary_tensor = tf.floor(random_tensor)
        output = tf.math.divide(inputs, keep_prob) * binary_tensor
        return output

batch_size, num_patches, dims = 2, 256, 768
drop_prob = .5

l = StochasticDepth(drop_prob)
xx = tf.random.normal((batch_size, num_patches, dims))
output = l(xx, training=True)
non_zeros = 1 - (tf.math.reduce_sum(tf.cast(output == 0, tf.int64))/(batch_size * num_patches * dims))

print('StochasticDepth Layer')
print('---------------------')
print(f'batch_size: {batch_size}, num_patches: {num_patches}, dims: {dims}')
print(f'\ndrop_prob: {drop_prob}')
print(f'\nInput: {xx.shape} --> {output.shape}')
print(f'\nOutput Drop Rate: {non_zeros}')